In [1]:
import pandas as pd

In [2]:
SHEET_ID = "1PznZPQOU1OFzB3QMCWxCoQYpYIbeU9FOQvWrR0wKHes"
SHEET_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv"

In [3]:
df = pd.read_csv(SHEET_URL, parse_dates=["fecha"], infer_datetime_format=True, dayfirst=True)

In [4]:
df = df.set_index("fecha")

In [5]:
df["activos"] = df["confirmados"] - df["recuperados"] - df["fallecidos"]

In [6]:
df["nuevos"] = df["confirmados"].diff()
df["nuevos"] = df["nuevos"].fillna(0)
df["nuevos"] = df["nuevos"].astype(int)

In [7]:
df["nuevos-mm7d"] = df["nuevos"].rolling(7).mean()
df["nuevos-mm7d"] = df["nuevos-mm7d"].fillna(0).astype(int)

In [8]:
print(f"Columnas: {df.columns.tolist()}")

Columnas: ['confirmados', 'recuperados', 'fallecidos', 'activos', 'nuevos', 'nuevos-mm7d']


In [ ]:
# Creamos los gráficos

from bokeh.io import output_notebook
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models import NumeralTickFormatter
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10
from bokeh.resources import INLINE
import datetime

In [ ]:
output_notebook()

In [ ]:
df.index.dtype

In [ ]:
df = df.loc["2020-07-01":]
df["fecha_str"] = df.index.astype(str)

source = ColumnDataSource(df)

TOOLTIPS = [("Fecha", "@{fecha_str}"),
            ("Casos activos", "@activos"),            
            ("Nuevos casos", "@nuevos"),
            ("Media móvil", "@{nuevos-mm7d}")]

In [ ]:
p = figure(plot_width=800, plot_height=540, name="Nuevos casos", x_axis_type='datetime', 
                 tooltips=TOOLTIPS)

In [ ]:
p.vbar(x="fecha", top="activos", source=source,
                 width=datetime.timedelta(days=1),
                 legend_label="Casos activos",
                 line_width=0,
                 color=Category10[4][1],
                 alpha=0.5,
#                 visible=False
      )

p.vbar(x="fecha", top="nuevos", source=source,
                 width=datetime.timedelta(days=1),
                 legend_label="Nuevos casos",
                 line_width=0,
                 color=Category10[4][0],
                 alpha=0.65,
      )

p.line(x="fecha", y="nuevos-mm7d", source=source,
                 legend_label="Nuevos casos (media móvil)",
                 line_width=1.75,
                 color=Category10[4][3],
                 alpha=0.65,
      )

p.legend.click_policy = 'hide'
p.legend.location = 'top_left'
p.legend.label_text_font_size = '8pt'
p.yaxis.formatter = NumeralTickFormatter(format='0,0')
p.xaxis.axis_label = "Fecha"
p.yaxis.axis_label = "Número de casos"
#p.title.text = "Reporte de casos de COVID-19 en el distrito de Saavedra (Provincia de Buenos Aires)"

In [ ]:
show(p)

In [ ]:
# A partir de acá se genera la página (index.html)

import panel as pn
import pytz

In [ ]:
pn.extension()

In [ ]:
ar = pytz.timezone('America/Argentina/Buenos_Aires')
now = datetime.datetime.now(ar)
dt_string = now.strftime("%d-%m-%Y %H:%M")

In [ ]:
header = pn.pane.Markdown(f"""
# COVID-19 -- Distrito de Saavedra (Provincia de Buenos Aires)

Datos obtenidos de la página de Instagram de la [Municipalidad de Saavedra](https://www.instagram.com/munisaavedrapigue/).

**Última actualización:** {dt_string}
""", width=800, margin=(0, 0, 10, 0)
)

In [ ]:
pn.Column(header, 
          p, 
          margin=(20, 10, 40, 40)
         ).save("index.html", 
                title="COVID-19 -- Distrito de Saavedra (Provincia de Buenos Aires)", 
                resources=INLINE)

In [ ]:
# Guardar el programa y para cargar a github:
# ./upload.sh
# usuario: agostinaf